# Урок 9. Языковое моделирование

## Задание

Разобраться с моделькой генерации текста. <br>
Cобрать самим или взять датасет с вебинара и обучить генератор.

## Решение

 * Добавить отслежевание точности. 
 * Увеличить к-во эпох обучения. хотябы до 200.

In [5]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
# Не показывать предупреждения.



In [2]:
path_to_file = 'evgenyi_onegin.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [5]:
text = text + text

In [6]:
# The unique characters in the file
#  Отсортируем все символы в датасете и удалим повотряющиеся. 
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [7]:
# Creating a mapping from unique characters to indices
#  Сделаем перевод номеров симоволов в тектст. 
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
# text_as_int, text, len(text_as_int), len(text)

In [9]:
text_as_int, text[:30], len(text_as_int), len(text)

(array([ 71, 110, 104, ..., 104, 121,   0]),
 'Александр Сергеевич Пушкин\n\n  ',
 573968,
 573968)

### train and target

In [10]:
# The maximum length sentence you want for a single input in characters

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) # ? Вот здесь не очень понятно. Зачем берут числа а не символы.

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [11]:
char_dataset.take(5)

<TakeDataset shapes: (), types: tf.int64>

In [12]:
# Выполним разбиение текста на последовательности длинной 100 символов.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# Выведем первые 5 батчей на экран.
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1] # Предсказываем по одному символу.
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

Print the first example input and target values:

In [15]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [6]:
tf.keras.layers.Dense?

Init signature: tf.keras.layers.Dense(*args, **kwargs)
Docstring:     
Just your regular densely-connected NN layer.

`Dense` implements the operation:
`output = activation(dot(input, kernel) + bias)`
where `activation` is the element-wise activation function
passed as the `activation` argument, `kernel` is a weights matrix
created by the layer, and `bias` is a bias vector created by the layer
(only applicable if `use_bias` is `True`). These are all attributes of
`Dense`.

Note: If the input to the layer has a rank greater than 2, then `Dense`
computes the dot product between the `inputs` and the `kernel` along the
last axis of the `inputs` and axis 1 of the `kernel` (using `tf.tensordot`).
For example, if input has dimensions `(batch_size, d0, d1)`,
then we create a `kernel` with shape `(d1, units)`, and the `kernel` operates
along axis 2 of the `input`, on every sub-tensor of shape `(1, 1, d1)`
(there are `batch_size * d0` such sub-tensors).
The output in this case will have shape `(

In [18]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [19]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [21]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [22]:
model = build_model(
            vocab_size=len(vocab),
            embedding_dim=embedding_dim,
            rnn_units=rnn_units,
            batch_size=BATCH_SIZE
        )

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         16768     
_________________________________________________________________
gru_2 (GRU)                  (None, None, 1024)        3545088   
_________________________________________________________________
gru_3 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
gru_4 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
dense_2 (Dense)              (None, None, 131)         134275    
Total params: 16,291,331
Trainable params: 16,291,331
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Посмотрим, что выдаст не тренированная сеть.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [ ]:
# Посотрим как выглядитя выходные значения из нейронной сети.

In [25]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 5.8422174e-04, -3.3047577e-04,  2.1373604e-04, ...,
         1.8112760e-04, -6.2016561e-04,  6.9306721e-04],
       [ 2.2242684e-03, -4.2144395e-04,  2.1331818e-03, ...,
        -5.1594386e-04, -1.8703066e-03,  2.3778772e-03],
       [ 3.2510846e-03, -4.7071901e-04,  5.5715214e-03, ...,
        -8.4109568e-05, -2.3435734e-03,  3.3656301e-03],
       ...,
       [ 3.4354196e-03, -1.2658591e-03,  1.0205108e-03, ...,
         2.3072492e-03, -9.5377618e-05,  3.5435005e-04],
       [ 5.7258797e-03, -1.6187496e-03,  3.3573562e-03, ...,
         1.6959890e-03, -1.8086392e-03,  1.5859769e-03],
       [ 7.2733308e-03, -6.9455983e-04,  4.4786404e-03, ...,
         1.0818078e-03, -3.4469906e-03,  2.2989344e-03]], dtype=float32)>

In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [27]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ир,\n                        Уездных матушек отрада,\n                        Приехал ротный командир;'

Next Char Predictions: 
 "В-чqdvObhQ iзнфzdцzи.mE5з'MQ32qщГыzа32z1A(F)QzF.(SWwГв\nЖaкиrAоЭiЮwтWз-u:gт;Sяч2ШSG!ОшНХ;ЮbySCПuЕвФос"


## Train the model

In [28]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# example_batch_loss = loss(target_example_batch, example_batch_predictions)
# print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
# print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [29]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

### Execute the training

In [31]:
EPOCHS = 20

In [32]:
history = model.fit(dataset,
                    epochs=EPOCHS,
                    callbacks=[checkpoint_callback]
                   )

Epoch 1/20
88/88 [==============================] - 44s 433ms/step - loss: 1.9826
Epoch 2/20
88/88 [==============================] - 39s 434ms/step - loss: 1.4418
Epoch 3/20
88/88 [==============================] - 39s 435ms/step - loss: 1.2999
Epoch 4/20
88/88 [==============================] - 39s 434ms/step - loss: 1.2102
Epoch 5/20
88/88 [==============================] - 39s 434ms/step - loss: 1.1233
Epoch 6/20
88/88 [==============================] - 39s 435ms/step - loss: 1.0429
Epoch 7/20
88/88 [==============================] - 39s 434ms/step - loss: 0.9577
Epoch 8/20
88/88 [==============================] - 39s 434ms/step - loss: 0.8630
Epoch 9/20
88/88 [==============================] - 39s 431ms/step - loss: 0.8538
Epoch 10/20
88/88 [==============================] - 38s 429ms/step - loss: 0.7484
Epoch 11/20
88/88 [==============================] - 38s 429ms/step - loss: 0.6220
Epoch 12/20
88/88 [==============================] - 38s 429ms/step - loss: 0.5125
Epoch 13/20
8

## Generate text

In [38]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [ ]:
# Востановим модель из кеша.

In [39]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [35]:
# model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         16768     
_________________________________________________________________
gru_5 (GRU)                  (None, None, 1024)        3545088   
_________________________________________________________________
gru_6 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
gru_7 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
dense_3 (Dense)              (None, None, 131)         134275    
Total params: 16,291,331
Trainable params: 16,291,331
Non-trainable params: 0
_________________________________________________________________


In [41]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [42]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже слямиюв    ши      это  понегоB1rarc3oтегой    .   бо     да- да                                      н?          сю              подла,               То,
      поны      зролюброретщегучтожни     рь          я,  Тьнь  бы,   кой,
         Гаемпи  жет,     это бо,
  з    де   том!
    и    всух Дранаругой,
        эть,
   набы..          драстырамисой И            Ивло       Я    прудразлишедаже
     гесящиховзгли                            раст.
  Ийт,
  Xь,
         нел
 У.
  вой           свол
